# Environment Setting Up

In [ ]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

In [ ]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

# 2. Base Model